## Sentence tokenizer with file slicing with size of 5000

This notebook helps to tokenize the paragraph into tokens of sentence using IO tokenizer. Also this has the functionality slice them into files of 5KB required as part of the further processing like translation and comprehend services.This note book also uses KMS encryption.

import io
import sys
import logging
import json
import boto3
import pandas as pd
from pandas.errors import ParserError

We would initialize the S3 object call the stringIO 

In [3]:
bucket = < enter your bucket name >
subfolder = < enter the subfolder>
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

# getting the execution role to download the file
role = get_execution_role()

s3_resource = boto3.resource('s3')
csv_buffer = StringIO()

We would write function to write files to S3 


In [5]:
def write_file(s3_key, file1, file2, count, serial_object, bucket):
    """
    this function used for writing the files to S3
    """
    s3_resource = boto3.resource('s3')
    if s3_key.endswith('.txt'):
       
        object_name = '{}'.format(s3_key.split('.txt')[0])
        state2_obj_name = object_name.replace(file1, file2)
        object_key = state2_obj_name + '_' + str(count + 1) + '.txt'

    s3_resource.Object(bucket, f"{object_key}").put(Body=serial_object,
                                                    ServerSideEncryption='aws:kms')

We would have function for splitting files into less than 5000 bytes size. As there could be residual sentecne of the size of 500 bytes we keep tolerance to 4500 bytes

In [25]:
def split_df4(S3_Key, df, file1, file2, bucket):
    """
    splitting the into chunks less than 5000 bytes
    """
    re_size = 5002
    df2 = pd.DataFrame()
    # as many of the functions may be having more than 500 bytes so the tolerance 5000 - 500
    rest = sys.getsizeof(df) // 4500

    if sys.getsizeof(df) < 5000:
        serial_object = df.to_csv(None, index=False, header=False, sep=' ')
        write_file(S3_Key, file1, file2, 0, serial_object, bucket)

    else:

        for i in range(rest):

            if i > 0:
                # working for the cases of df being on the higher side of 5000
                re_size = sys.getsizeof(df2)
                df = df2
                df2 = pd.DataFrame()

            else:
                # looping for the file size
                sys.getsizeof(df)
                last_row = df.tail(1)
                df2 = df2.append(last_row, ignore_index=True)

            while re_size > 5000:
                df = df.dropna()
                last_row = df.tail(1)
                df2 = df2.append(last_row, ignore_index=True)
                re_size = sys.getsizeof(df)
                df = df[: -1]
            df2 = df2.iloc[::-1]
            df2 = df2.reset_index(drop=True)
            serial_object = df.to_csv(None, index=False, header=False, sep=' ')
            if sys.getsizeof(serial_object) > 62:
                write_file(S3_Key, file1, file2, i, serial_object, bucket)
            # print("df"+str(i),df )
        df2 = df2[: -1]
        df = pd.DataFrame()  # do not remove this required for reset of dataframe for every loop
        serial_object = df2.to_csv(None, index=False, header=False, sep=' ')
        
        # in the dataframe used we had the column name size of 62 bytes so zero entry file would have min
        # if there are any un treated junk characterds are remaining we are removing it
        if 62 < sys.getsizeof(serial_object) < 5000:
            write_file(S3_Key, file1, file2, i + 1, serial_object, bucket)
        i=0
        df2 = pd.DataFrame()  # do not remove this required for reset of dataframe for every loop


In [ ]:
def main():
    """
       main function
    """
    conn = boto3.client('s3')
    contents = conn.list_objects(Bucket=bucket, Prefix=sub_folder)['Contents']
    file_1 = input_prefix + "/" + run_date
    file_2 = output_prefix + "/" + run_date
    for file in contents:
        # print(f['Key'])
        # we can select a file to do tokenization as 1613953.txt
        s3_key = file['Key']
        if s3_key == sub_folder:
            continue
        s3_object = conn.get_object(Bucket=bucket, Key=s3_key)
        try:
            df = pd.read_csv(io.BytesIO(s3_object['Body'].read()), sep='delimiter', engine='python')
        except ParserError:
            df = pd.read_csv(io.BytesIO(s3_object['Body'].read()), error_bad_lines=False,
                             names=['token'], encoding='utf-8')

        # cleaning the data frame
        # print(df)
        split_df4(s3_key, df, file_1, file_2, bucket)


if __name__ == "__main__":
    main()